In [11]:
import warnings

warnings.filterwarnings("ignore")
import xalpha as xa

# 网格回测

在这一笔记中，我们将展示如何使用xalpha进行网格回测。网格回测是一种简单的策略，它在价格波动的时候，以固定的间隔买入或卖出，从而在价格波动中获利。我们主要考虑网格间距和比例等不同的超参对于真实收益的影响。

In [5]:
def generate_settings(
    high_price, low_price, price_ratio, amount_ratio, remain_ratio=None
):
    """
    generate price and amount list for grid policy
    :param high_price: float, the highest price
    :param low_price: float, the lowest price
    :param price_ratio: float, the ratio between two adjacent prices
    :param amount_ratio: float, the ratio between two adjacent amounts
    :param remain_ratio: float, the ratio of the remaining amount to sell (留利润)
    """
    if remain_ratio is None:
        remain_ratio = price_ratio
    prices = [high_price]
    while prices[-1] > low_price:
        prices.append(round(prices[-1] * price_ratio, 3))
    inamount = [10000]
    for _ in range(len(prices) - 2):
        inamount.append(inamount[-1] * amount_ratio)
    outamount = [10000 * remain_ratio]
    for i in range(len(prices) - 2):
        outamount.append(outamount[-1] * amount_ratio)

    return prices, inamount, outamount

In [7]:
# 对于传媒指数基金，我们可以生成如下的网格策略

prices, inamount, outamount = generate_settings(0.9, 0.48, 0.95, 1.0)

# 返回的三个列表分别是价格列表，买入金额列表，卖出金额列表
prices, inamount, outamount

([0.9,
  0.855,
  0.812,
  0.771,
  0.732,
  0.695,
  0.66,
  0.627,
  0.596,
  0.566,
  0.538,
  0.511,
  0.485,
  0.461],
 [10000,
  10000.0,
  10000.0,
  10000.0,
  10000.0,
  10000.0,
  10000.0,
  10000.0,
  10000.0,
  10000.0,
  10000.0,
  10000.0,
  10000.0],
 [9500.0,
  9500.0,
  9500.0,
  9500.0,
  9500.0,
  9500.0,
  9500.0,
  9500.0,
  9500.0,
  9500.0,
  9500.0,
  9500.0,
  9500.0])

In [12]:
# 现在我们开始测试这个策略

bt = xa.backtest.Grid(
    start="20240101",
    end="20241231",
    code="SH512980",
    prices=prices,
    inamount=inamount,
    outamount=outamount,
)
bt.backtest()

In [13]:
# 生成回测持仓报告

bt.get_current_mul().combsummary("20241231")

,基金名称,基金代码,当日净值,单位成本,持有份额,基金现值,基金总申购,历史最大占用,基金持有成本,基金分红与赎回,换手率,基金收益总额,投资收益率
0,传媒ETF,SH512980,0.774,0.4355,33000.0,25542.0,189970.0,60901.0,14372.0,175598.0,3.009575,11170.0,18.3412
1,总计,total,NaN,NaN,NaN,25542.0,189970.0,60885.5,14372.0,175598.0,2.501956,11170.0,18.3459


In [14]:
bt.get_current_mul().combsummary("20241231")["投资收益率"].iloc[0]

18.3412

In [16]:
bt.get_current_mul().xirrrate("20241231")

0.297174042371061

In [25]:
# 现在我们进行不同网格超参策略的评估


def evaluate(
    code,
    high_price,
    low_price,
    price_ratio,
    amount_ratio,
    remain_ratio=None,
    start="20240101",
    end="20241231",
):
    prices, inamount, outamount = generate_settings(
        high_price, low_price, price_ratio, amount_ratio, remain_ratio
    )
    bt = xa.backtest.Grid(
        start=start,
        end=end,
        code=code,
        prices=prices,
        inamount=inamount,
        outamount=outamount,
    )
    bt.backtest()
    mul = bt.get_current_mul()
    return round(mul.combsummary("20241231")["投资收益率"].iloc[0], 2), round(
        mul.xirrrate("20241231") * 100, 2
    )

In [32]:
# 不同网格间距的收益
for d in [0.99, 0.98, 0.97, 0.96, 0.95, 0.94, 0.93, 0.92, 0.91, 0.9, 0.85, 0.8]:
    print(d, evaluate("SH512980", 0.9, 0.48, d, 1))

# 我们看到不同网格间距对收益的影响不是特别大并且不单调

0.99 (16.14, 28.13)
0.98 (18.9, 32.35)
0.97 (18.95, 32.2)
0.96 (18.3, 29.0)
0.95 (18.34, 29.72)
0.94 (17.29, 26.51)
0.93 (17.71, 27.08)
0.92 (19.89, 29.47)
0.91 (20.14, 31.86)
0.9 (21.33, 32.1)
0.85 (19.36, 25.37)
0.8 (28.06, 42.65)


In [34]:
# 我们测试这一结果相对其他超参变化的鲁棒性，如回测时间段, 每网买入量的梯度等

for d in [0.99, 0.98, 0.97, 0.96, 0.95, 0.94, 0.93, 0.92, 0.91, 0.9, 0.85, 0.8]:
    print(d, evaluate("SH512980", 0.9, 0.48, d, 1, start="20230101", end="20241231"))

for d in [0.99, 0.98, 0.97, 0.96, 0.95, 0.94, 0.93, 0.92, 0.91, 0.9, 0.85, 0.8]:
    print(d, evaluate("SH512980", 0.9, 0.48, d, 1.05))

0.99 (41.91, 50.1)
0.98 (46.33, 52.86)
0.97 (47.62, 53.2)
0.96 (46.91, 48.82)
0.95 (45.88, 48.89)
0.94 (43.47, 43.61)
0.93 (42.82, 43.3)
0.92 (47.51, 46.02)
0.91 (47.99, 48.62)
0.9 (50.35, 49.41)
0.85 (52.6, 43.4)
0.8 (51.83, 55.55)
0.99 (15.46, 42.17)
0.98 (18.75, 39.27)
0.97 (18.82, 36.44)
0.96 (18.73, 32.0)
0.95 (18.83, 32.51)
0.94 (17.54, 28.09)
0.93 (18.1, 28.71)
0.92 (20.3, 30.94)
0.91 (20.25, 33.14)
0.9 (21.68, 33.4)
0.85 (19.74, 26.07)
0.8 (28.61, 44.12)


In [35]:
# 换个信息技术的标的

for d in [0.99, 0.98, 0.97, 0.96, 0.95, 0.94, 0.93, 0.92, 0.91, 0.9, 0.85, 0.8]:
    print(d, evaluate("SZ159939", 0.72, 0.35, d, 1))

0.99 (16.91, 29.01)
0.98 (19.38, 31.98)
0.97 (18.87, 31.4)
0.96 (19.45, 31.25)
0.95 (19.28, 30.87)
0.94 (20.71, 33.25)
0.93 (20.57, 31.19)
0.92 (21.32, 32.06)
0.91 (23.89, 36.36)
0.9 (21.01, 33.48)
0.85 (25.83, 36.22)
0.8 (23.05, 26.23)


In [45]:
# 更合理的封闭系统收益率评估, 考虑剩余现金的收益


def evaluate_fix(
    code,
    high_price,
    low_price,
    price_ratio,
    amount_ratio,
    remain_ratio=None,
    start="20240101",
    end="20241231",
):
    prices, inamount, outamount = generate_settings(
        high_price, low_price, price_ratio, amount_ratio, remain_ratio
    )
    bt = xa.backtest.Grid(
        start=start,
        end=end,
        code=code,
        prices=prices,
        inamount=inamount,
        outamount=outamount,
    )
    bt.backtest()
    mul = bt.get_current_mul()
    totmoney = mul.combsummary("20241231")["历史最大占用"].iloc[0]
    mulfix = bt.get_current_mulfix(totmoney=totmoney + 1.0)
    return mulfix.combsummary("20241231")["投资收益率"].iloc[0]

In [43]:
mulfix = evaluate_fix("SZ159939", 0.72, 0.35, 0.95, 1)

In [46]:
for d in [0.99, 0.98, 0.97, 0.96, 0.95, 0.94, 0.93, 0.92, 0.91, 0.9, 0.85, 0.8]:
    print(d, evaluate_fix("SZ159939", 0.72, 0.35, d, 1))

0.99 18.4835
0.98 20.8728
0.97 20.3825
0.96 20.8855
0.95 20.6979
0.94 22.1393
0.93 21.8617
0.92 22.5938
0.91 25.1978
0.9 22.4241
0.85 26.9288
0.8 23.5061


结论就是不同的网格宽度对应的收益率大体类似，甚至更宽的网格收益情况更好，因此选取较低频的网格宽度即可。

下面我们评估越便宜买的越多的网格策略是否有超额收益。

In [47]:
for d in [1.0, 1.02, 1.04, 1.06, 1.08, 1.1, 1.15, 1.2]:
    print(d, evaluate_fix("SZ159939", 0.72, 0.35, 0.94, d))

1.0 22.1393
1.02 21.8901
1.04 21.2032
1.06 20.5496
1.08 17.4569
1.1 21.174
1.15 19.3504
1.2 20.0333


整体结论可能也是区别不大，对于网格具体超参调优可以根据自己的投资标的和策略自行回测，但警惕过拟合陷阱，可能本文的主要结论就是这些超参的调整对于最后的收益都区别不是特别大。